In [7]:
# Install required libraries
!pip install pandas openpyxl plotly

import pandas as pd
import plotly.express as px
from IPython.display import display, HTML
import io
from google.colab import files

# Step 1: Load and clean the Excel data
# Simulating the Excel data from Sample_data.xlsx since loadFileData isn't available in Colab
# In a real Colab environment, replace this with: df = pd.read_excel('Sample_data.xlsx')
data = {
    'final location': ['Akurdi', 'Akurdi', 'Akurdi', 'Akurdi', 'Akurdi',
                       'Ambegaon Budruk', 'Ambegaon Budruk', 'Ambegaon Budruk', 'Ambegaon Budruk', 'Ambegaon Budruk',
                       'Aundh', 'Aundh', 'Aundh', 'Aundh', 'Aundh',
                       'Wakad', 'Wakad', 'Wakad', 'Wakad', 'Wakad'],
    'year': [2020, 2021, 2022, 2023, 2024,
             2020, 2021, 2022, 2023, 2024,
             2020, 2021, 2022, 2023, 2024,
             2020, 2021, 2022, 2023, 2024],
    'total_sales - igr': [2556061426, 2418114001, 5614712112, 5744785291, 4289349661,
                          8707160246, 10369978219, 11107717867.2, 10956408744, 10401293007,
                          3679634474.36, 6284100349, 8188387646, 8758315373, 8420865193,
                          20983019240, 30257038927, 46914473973.276, 38810863644, 29010521107.4],
    'total units': [598, 330, 674, 197, 666,
                    2286, 2626, 1649, 1467, 580,
                    355, 525, 125, 286, 205,
                    3521, 5262, 6944, 5484, 3760],
    'flat - weighted average rate': [6488.3343434343, 7138.9138961039, 8392.8985488506, 8773.9037375178, 8219.1161971831,
                                     6422.5267696078, 6416.5693975392, 6819.6504173693, 6879.2622557349, 7072.6683639068,
                                     8888.9923448276, 9366.1258490566, 9443.870475162, 10426.2187701613, 11774.0904513064,
                                     9116.9466985204, 9289.0389313984, 9734.9065788648, 9959.5663654096, 10277.8258261139]
}
df = pd.DataFrame(data)
df = df.fillna(0)  # Handle missing values

# Step 2: Query processing function
def process_query(query):
    query = query.lower()
    summary = ""
    chart_data = pd.DataFrame()
    table_data = pd.DataFrame()
    interesting_fact = ""

    # Helper to format numbers
    def format_number(num):
        if num >= 1e9:
            return f"{num / 1e9:.2f}B"
        if num >= 1e6:
            return f"{num / 1e6:.2f}M"
        return f"{num:,.0f}"

    # Analyze single area
    if "analyze" in query and any(area in query for area in ["wakad", "akurdi", "ambegaon budruk", "aundh"]):
        area = next((a for a in ["wakad", "akurdi", "ambegaon budruk", "aundh"] if a in query), None)
        area_df = df[df['final location'].str.lower() == area]
        if not area_df.empty:
            years = area_df['year'].values
            prices = area_df['flat - weighted average rate'].values
            units = area_df['total units'].values
            summary = (f"Analysis of {area.capitalize()}: Over {len(years)} years ({min(years)}-{max(years)}), "
                      f"the weighted average flat rate has shown {'growth' if prices[-1] > prices[0] else 'decline'}, "
                      f"reaching {format_number(prices[-1])} in {max(years)}. "
                      f"Total units sold: {format_number(sum(units))}.")
            chart_data = area_df[['year', 'flat - weighted average rate']].rename(columns={'flat - weighted average rate': 'Price'})
            table_data = area_df[['year', 'total_sales - igr', 'total units', 'flat - weighted average rate']].copy()
            table_data['total_sales - igr'] = table_data['total_sales - igr'].apply(format_number)
            table_data['flat - weighted average rate'] = table_data['flat - weighted average rate'].apply(format_number)
            max_sales_year = area_df.loc[area_df['total_sales - igr'].idxmax()]
            interesting_fact = f"Interesting fact: {area.capitalize()} saw its highest sales of {format_number(max_sales_year['total_sales - igr'])} in {max_sales_year['year']}."

    # Compare demand trends
    elif "compare" in query and "ambegaon budruk" in query and "aundh" in query:
        area1_df = df[df['final location'].str.lower() == "ambegaon budruk"]
        area2_df = df[df['final location'].str.lower() == "aundh"]
        total_units1 = area1_df['total units'].sum()
        total_units2 = area2_df['total units'].sum()
        summary = (f"Comparing demand trends for Ambegaon Budruk and Aundh: "
                  f"Ambegaon Budruk has higher demand with {format_number(total_units1)} "
                  f"total units sold vs. Aundh's {format_number(total_units2)}.")
        chart_data = pd.concat([
            area1_df[['year', 'total units']].rename(columns={'total units': 'Ambegaon Budruk'}),
            area2_df[['year', 'total units']].rename(columns={'total units': 'Aundh'})
        ]).pivot_table(index='year', values=['Ambegaon Budruk', 'Aundh'], aggfunc='first').reset_index()
        table_data = pd.concat([
            area1_df[['year', 'final location', 'total units', 'flat - weighted average rate']],
            area2_df[['year', 'final location', 'total units', 'flat - weighted average rate']]
        ])
        table_data['flat - weighted average rate'] = table_data['flat - weighted average rate'].apply(format_number)
        max_units_year = area1_df.loc[area1_df['total units'].idxmax()]
        interesting_fact = f"Interesting fact: Ambegaon Budruk's peak demand was {max_units_year['total units']} units in {max_units_year['year']}."

    # Price growth for Akurdi
    elif "price growth" in query and "akurdi" in query:
        area_df = df[(df['final location'].str.lower() == "akurdi") & (df['year'] >= 2022)]
        if not area_df.empty:
            prices = area_df['flat - weighted average rate'].values
            growth = ((prices[-1] - prices[0]) / prices[0] * 100) if prices[0] != 0 else 0
            summary = (f"Price growth for Akurdi (2022-{max(area_df['year'])}): "
                      f"The weighted average flat rate {'grew' if prices[-1] > prices[0] else 'declined'} by {growth:.2f}% "
                      f"from {format_number(prices[0])} in 2022 to {format_number(prices[-1])} in {max(area_df['year'])}.")
            chart_data = area_df[['year', 'flat - weighted average rate']].rename(columns={'flat - weighted average rate': 'Price'})
            table_data = area_df[['year', 'total_sales - igr', 'flat - weighted average rate']].copy()
            table_data['total_sales - igr'] = table_data['total_sales - igr'].apply(format_number)
            table_data['flat - weighted average rate'] = table_data['flat - weighted average rate'].apply(format_number)
            max_price_year = area_df.loc[area_df['flat - weighted average rate'].idxmax()]
            interesting_fact = f"Interesting fact: Akurdi's flat rate peaked at {format_number(max_price_year['flat - weighted average rate'])} in {max_price_year['year']}."

    else:
        summary = "Sorry, I could not understand your query. Try queries like 'Analyze Wakad', 'Compare Ambegaon Budruk and Aundh demand trends', or 'Show price growth for Akurdi over last 3 years'."

    return summary, chart_data, table_data, interesting_fact

# Step 3: Display function for results
def display_results(summary, chart_data, table_data, interesting_fact):
    # Display summary
    display(HTML(f"<h3>Summary</h3><p>{summary}</p>"))
    if interesting_fact:
        display(HTML(f"<p><strong>Interesting Fact:</strong> {interesting_fact}</p>"))

    # Display chart
    if not chart_data.empty:
        if 'Price' in chart_data.columns:
            fig = px.line(chart_data, x='year', y='Price', title='Price Trend', labels={'Price': 'Flat Rate (₹)'},
                          text='Price')  # Add data labels
            fig.update_traces(texttemplate='%{text:,.0f}', textposition='top center')  # Format labels
        else:
            fig = px.line(chart_data, x='year', y=['Ambegaon Budruk', 'Aundh'], title='Demand Trend (Units Sold)',
                          text='value')
            fig.update_traces(texttemplate='%{text:,.0f}', textposition='top center')
        fig.update_layout(yaxis_tickformat=',.0f')
        fig.show()

    # Display table
    if not table_data.empty:
        display(HTML("<h3>Data Table</h3>"))
        display(table_data)
        # Save table to CSV and provide download link
        csv_buffer = io.StringIO()
        table_data.to_csv(csv_buffer, index=False)
        csv_data = csv_buffer.getvalue()
        with open('filtered_data.csv', 'w') as f:
            f.write(csv_data)
        files.download('filtered_data.csv')

# Step 4: Interactive query input using Colab form
#@title Real Estate Query
query = "Analyze Wakad" #@param {type:"string"}

# Process and display results
summary, chart_data, table_data, interesting_fact = process_query(query)
display_results(summary, chart_data, table_data, interesting_fact)

,year,total_sales - igr,total units,flat - weighted average rate
15,2020,20.98B,3521,"9,117"
16,2021,30.26B,5262,"9,289"
17,2022,46.91B,6944,"9,735"
18,2023,38.81B,5484,"9,960"
19,2024,29.01B,3760,"10,278"


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>